In [ ]:
import lukenet
import lukenet_helpers as helpers
import matplotlib.pyplot as plt
import numpy as np

#### Selecting Input Data for the LukeNet Model

We start by extracting outputs from the **DALI benchmark model**. Here are the details:

- **DALI Benchmark Models** are located in the folder `benchmark/`.
- The suffix of each model file represents the chemical timescale.
  - For example: `benchmark_dali_1e6` corresponds to model data for **1 Myr**.
- The **DALI models** are run on a **100 x 80 grid** (r x z).
  - Select a grid cell below. The helper function will then print out relevant properties.
  - Note that the values are indices (not distances in AU!)

In [ ]:
r_cell, z_cell = 70, 20
helpers.print_dali_cell('benchmark/benchmark_dali_1e6/out.dat', r_cell, z_cell)

#### Copying Data into the LukeNet Input File

Now copy and paste the entire block of data above into the LukeNet input file (`lukenet_inputs.py`):

- The data is already formatted exactly as needed when printed by the helper function.
- Simply copy and paste the entire block into the input file, below where it says `# Extracted from DALI model...`
- The values that are already in the input file are for cells (70,20), which are a default for testing, but you can change to any you want

**Important:**  
Do **NOT** change the four inputs below the DALI parameters (Zeta_CR, pah_ism, t_chem, netork) — these remain fixed across all models.


#### Running the LukeNet Model

After saving the input file, you can run the LukeNet model. Typical usage is shown below:

- A progress bar should be displayed.
- Using the default values above (r=70, z=20), the model should finish in ~10 seconds. Other sets of input parameters will lead to variations in compute time.

- The output is saved to `result`, which is a dictionary containing key information such as:
  - Abundances
  - Reaction rates
  - Other important data

In [ ]:
# Run LukeNet()
network = lukenet.Lukenet()
network.init_lukenet()
result = network.solve_network()

#### Plotting the Results and Comparing to DALI Benchmark Models

Now it's time to plot the results and compare them to the DALI benchmark models:

1. **Import the Data** from the DALI models (as shown in the first block below).
   - This may take 1-2 minutes.
2. **Plot Abundances vs. Time** (second block below):
   - DALI data will be represented as **scatter points**.
   - LukeNet data will be shown as **solid lines**.
   - For the default values `(70,20)`, you should find excellent agreement between DALI/LukeNet
   
**Note:**  
- Only a subset of all species is displayed for clarity.  
- You can edit the species list to include any species you want, **as long as they are included in the chemical network**.


In [ ]:
# Import DALI benchmark data
dali_1em4 = helpers.read_outdat('benchmark/benchmark_dali_1e-4/out.dat')
dali_1em3 = helpers.read_outdat('benchmark/benchmark_dali_1e-3/out.dat')
dali_1em2 = helpers.read_outdat('benchmark/benchmark_dali_1e-2/out.dat')
dali_1em1 = helpers.read_outdat('benchmark/benchmark_dali_1e-1/out.dat')
dali_1e0  = helpers.read_outdat('benchmark/benchmark_dali_1e0/out.dat')
dali_1e1  = helpers.read_outdat('benchmark/benchmark_dali_1e1/out.dat')
dali_1e2  = helpers.read_outdat('benchmark/benchmark_dali_1e2/out.dat')
dali_1e3  = helpers.read_outdat('benchmark/benchmark_dali_1e3/out.dat')
dali_1e4  = helpers.read_outdat('benchmark/benchmark_dali_1e4/out.dat')
dali_1e5  = helpers.read_outdat('benchmark/benchmark_dali_1e5/out.dat')
dali_1e6  = helpers.read_outdat('benchmark/benchmark_dali_1e6/out.dat')

In [ ]:
###################
# Plot abundances #
###################

# Species we want to plot...
species = ['H', 'H2', 'C', 'N', 'O', 'CO', 'H2O', 'CH4', 'N2', 'JH2O', 'PAH0', 'PAH+', 'PAH-', 'PAH_H', 'CH']


# DALI models
model_list = [dali_1em4, dali_1em3, dali_1em2, dali_1em1, dali_1e0, dali_1e1, dali_1e2, dali_1e3, dali_1e4, dali_1e5, dali_1e6]
time_dali  = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]

abu_list = []
for model in model_list:
    abu_list_temp = [(np.array(model[specie])[r_cell, z_cell] / np.array(model['n_gas'])[r_cell, z_cell]) for specie in species]
    abu_list.append(abu_list_temp)
    
dali_abundances = np.array(abu_list)


# Set up colors for the plot
default_colors  = plt.rcParams["axes.prop_cycle"].by_key()["color"] * 30 
color_list      = default_colors[:len(species)]


# Plotting
fig = plt.figure(figsize=(8,6))

for i in range(0, len(species)):
    idx_specie = network.species.name.index(species[i])
    # LukeNet
    plt.loglog(result['time'] / network.parameters.yr_sec, result['abundances'][idx_specie]/network.gas.n_gas, alpha=0.6, color=color_list[i])
    # DALI
    plt.scatter(time_dali, dali_abundances[:, i], alpha=0.6, color=color_list[i], s=20)
    
    plt.annotate(network.species.name[idx_specie], (1.5*result['time'][-1] / network.parameters.yr_sec, (result['abundances'][idx_specie][-1]/network.gas.n_gas)), fontsize=10, color=color_list[i], alpha=0.7)
    plt.annotate(network.species.name[idx_specie], (0.3*result['time'][0] / network.parameters.yr_sec, (result['abundances'][idx_specie][0]/network.gas.n_gas)), fontsize=8, color=color_list[i], alpha=0.7)

plt.xlabel('Time (years)', fontsize=12)
plt.ylabel('Abundance (X/H)', fontsize=12)

plt.xlim(1e-3, 5e6)
plt.ylim(1e-12, 3)
plt.tight_layout()
plt.show()